# ABC XYZ

# ПРОГНОЗИРОВАНИЕ

# № 2

In [1]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 hours 19 mins
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.2
H2O_cluster_version_age:,2 months and 2 days
H2O_cluster_name:,H2O_from_python_unknownUser_0c726b
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.075 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
#file = '/content/drive/MyDrive/Colab Notebooks/Diplom/df4eda2clean2.csv'
file = 'df2prep4regressionV7.csv'
df = h2o.import_file(file)
df = df.drop('C1')
df = df.drop('Revenue_cat')
df

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Zip,Manufacturer,MonthNo,CategoryInSegment,abc_xyz_class
6111,Natura,12,UrbanConvenience,BY
92879,Pirum,12,RuralProductivity,BY
32901,VanArsdel,12,UrbanModeration,AY
84770,Currus,12,RuralSelect,AZ
95380,Abbas,12,UrbanModeration,BZ
14559,Natura,12,YouthYouth,BY
16301,Natura,12,YouthYouth,BY
48439,Quibus,12,RuralSelect,BX
21787,VanArsdel,12,UrbanModeration,AY
7503,Currus,12,MixAll Season,BY


In [3]:
train, valid, unseen = df.split_frame(ratios=[0.7, 0.15])

In [4]:
train

Zip,Manufacturer,MonthNo,CategoryInSegment,abc_xyz_class
6111,Natura,12,UrbanConvenience,BY
92879,Pirum,12,RuralProductivity,BY
32901,VanArsdel,12,UrbanModeration,AY
16301,Natura,12,YouthYouth,BY
48439,Quibus,12,RuralSelect,BX
21787,VanArsdel,12,UrbanModeration,AY
7503,Currus,12,MixAll Season,BY
60073,Natura,12,UrbanConvenience,AY
70578,VanArsdel,12,UrbanConvenience,AX
32164,Natura,12,UrbanModeration,AY


In [5]:
valid

Zip,Manufacturer,MonthNo,CategoryInSegment,abc_xyz_class
84770,Currus,12,RuralSelect,AZ
33126,Natura,12,UrbanModeration,AY
30901,Natura,12,UrbanExtreme,AY
16412,Aliqui,12,UrbanExtreme,AY
56097,VanArsdel,12,UrbanModeration,AX
25312,Currus,12,UrbanExtreme,AY
30559,Currus,12,UrbanExtreme,AY
93648,Currus,12,UrbanExtreme,AY
11931,VanArsdel,12,UrbanModeration,AX
51041,VanArsdel,12,UrbanModeration,AX


In [6]:
unseen

Zip,Manufacturer,MonthNo,CategoryInSegment,abc_xyz_class
95380,Abbas,12,UrbanModeration,BZ
14559,Natura,12,YouthYouth,BY
6450,Natura,12,UrbanModeration,AY
10032,Aliqui,12,UrbanExtreme,AX
19064,Natura,12,UrbanExtreme,AY
97140,Aliqui,12,UrbanExtreme,AY
94065,Aliqui,12,UrbanExtreme,AY
48656,VanArsdel,12,UrbanModeration,AX
30534,VanArsdel,12,UrbanModeration,AX
89117,Currus,12,UrbanExtreme,AY


In [7]:
y = "abc_xyz_class"
x=train.columns
x.remove(y)

# Тренировка модели

In [8]:
aml = H2OAutoML(max_models = 10, seed = 1)
aml.train(x = x, y = y,  training_frame = train, validation_frame = valid)

AutoML progress: |
17:58:20.463: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

██████████████████████████████████████████████████████████████
18:45:31.43: GBM_1_AutoML_2_20230927_175820 [GBM def_5] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_1_AutoML_2_20230927_175820_cv_1.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (143.8 KB per tree x 10000 > 735.6 MB) - try decreasing ntrees and/or max_depth or increasing min_rows!


Failed polling AutoML progress log: Unexpected HTTP error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
 (failed) 100%


OSError: Job with key $03017f00000132d4ffffffff$_b07690d2d1ea9f089e97c71429384ce9 failed with an exception: java.lang.OutOfMemoryError: Java heap space
stacktrace: 
java.lang.OutOfMemoryError: Java heap space


In [ ]:
lb = aml.leaderboard
lb

In [ ]:
# Get model ids for all models in the AutoML Leaderboard
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])
model_ids

In [ ]:
# Get the "All Models" Stacked Ensemble model
se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
se

In [ ]:
# Get the Stacked Ensemble metalearner model
metalearner = h2o.get_model(se.metalearner()['name'])

In [ ]:
metalearner.coef_norm()

In [ ]:
%matplotlib inline
#metalearner.std_coef_plot()

# Что с важностью признаков

In [ ]:
for model_id in model_ids:
  try:
    print(f"Model ID: {model_id}")
    aml_unit = h2o.get_model(model_id)
    aml_unit.varimp_plot()
  except:
    print(f"{model_id} - ERROR PLOTTING") 

# Что с переобученносью

In [ ]:
import pandas as pd
from sklearn.metrics import r2_score

for model_id in model_ids:
  try:
    print(f"Model ID: {model_id}")
    aml_unit = h2o.get_model(model_id)
    perf = aml_unit.model_performance(valid)
    print(perf)
    print(f"R2 cross validation: {aml_unit.r2()}")
    print(f"R2 test validation: {aml_unit.r2(valid=True)}")
    # прогноз лидирующей моделю на валидационной выборке
    val_pred=aml_unit.predict(unseen).as_data_frame()
    # из валидационного датасеат возбму столбец Stay и сконвертиую его в пандас датафрейм
    val_test = unseen['abc_xyz_class'].as_data_frame()
    r2_unseen = r2_score(val_test, val_pred)
    print(f"R2 unseen dataset validation: {r2_unseen}")
    print("-------------------\n-------------------")
  except:
    print(f"{model_id} - ERROR CALCULATE PERFOMANCE")

# Графии кривых обучения

In [ ]:
for model_id in model_ids:
  try:
    learning_curve_plot = h2o.get_model(model_id).learning_curve_plot()
  except:
    print(model_id, "- ERROR PLOT")

# Сохранение лучше модели

Довольно стаблильный показатель по Р-квадарт

In [ ]:
#besy_model_id = 'StackedEnsemble_BestOfFamily_1_AutoML_4_20230921_114817'

In [ ]:
# Сохранение лучшей модели
#besy_model = h2o.get_model(besy_model_id)
#h2o.save_model(besy_model, path = "./models")